In [143]:
from time import sleep
from tqdm import tqdm

import numpy as np
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials

from keys import GOOG_API_KEY

In [144]:
## Load dataframes
# Polling averages
pres = pd.read_csv('averages/presidential_gen_approval.csv')
pres_issue = pd.read_csv('averages/presidential_issue_approvals.csv')
pres_rv = pd.read_csv('averages/presidential_RV_approval.csv')
generic_ballot = pd.read_csv('averages/generic_ballot.csv')

# Polls display tables
pres_polls = pd.read_csv('transformed_tables/gen_approval_polls.csv')
issue_polls = pd.read_csv('transformed_tables/issue_approval_polls.csv')
gb_polls = pd.read_csv('transformed_tables/generic_ballot_polls_disp.csv')

In [145]:
# Define the scope
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

# Authenticate with the service account
credentials = Credentials.from_service_account_file(
    GOOG_API_KEY,
    scopes=scopes
)

# Authorize gspread
client = gspread.authorize(credentials)

## Polling Averages

In [146]:
spreadsheet = client.open("snoutcounter-data")
worksheet = spreadsheet.worksheet("approval-stats")
worksheet.clear()

set_with_dataframe(worksheet, pres.reset_index())

In [147]:
worksheet = spreadsheet.worksheet("approval_stats_RV")
worksheet.clear()

set_with_dataframe(worksheet, pres_rv.reset_index())

In [148]:
issues = ['economy', 'immigration', 'inflation', 'trade_tariffs', 'foreign_policy', 'healthcare', 'crime']
issue_suffixes = ['econ', 'imm', 'infl', 'trade', 'fp', 'healthcare', 'crime']

for iss, suf in tqdm(zip(issues, issue_suffixes)):
    worksheet = spreadsheet.worksheet(f"approval_stats_{suf}")
    worksheet.clear()

    issue_df = pres_issue[pres_issue['issue'] == iss]
    set_with_dataframe(worksheet, issue_df.reset_index())

7it [00:06,  1.05it/s]


In [149]:
## Net issue approval
net_issues = pd.pivot_table(pres_issue[pres_issue['issue'].isin(issues)], 
                            values=['net', 'net_lower_ci', 'net_upper_ci'], index=['end_date'], columns=['issue'],
                           aggfunc="first").reset_index()
net_issues.columns = ['_'.join(col).strip() for col in net_issues.columns.values]
net_issues.head()

,end_date_,net_crime,net_economy,net_foreign_policy,net_healthcare,net_immigration,net_inflation,net_trade_tariffs,net_lower_ci_crime,net_lower_ci_economy,...,net_lower_ci_immigration,net_lower_ci_inflation,net_lower_ci_trade_tariffs,net_upper_ci_crime,net_upper_ci_economy,net_upper_ci_foreign_policy,net_upper_ci_healthcare,net_upper_ci_immigration,net_upper_ci_inflation,net_upper_ci_trade_tariffs
0,2025-01-21,0.0,8.058340,4.05834,0.000000,10.058340,-3.94166,0.05834,0.0,8.058340,...,10.058340,-3.94166,0.05834,0.0,8.058340,4.05834,0.000000,10.05834,-3.94166,0.05834
1,2025-01-22,0.0,8.058340,4.05834,0.000000,10.058340,-3.94166,0.05834,0.0,8.058340,...,10.058340,-3.94166,0.05834,0.0,8.058340,4.05834,0.000000,10.05834,-3.94166,0.05834
2,2025-01-23,0.0,2.893584,4.05834,-13.651576,3.553815,-3.94166,0.05834,0.0,-4.212634,...,-5.395801,-3.94166,0.05834,0.0,9.999803,4.05834,-13.651576,12.50343,-3.94166,0.05834
3,2025-01-24,0.0,2.893584,4.05834,-13.651576,3.553815,-3.94166,0.05834,0.0,-4.212634,...,-5.395801,-3.94166,0.05834,0.0,9.999803,4.05834,-13.651576,12.50343,-3.94166,0.05834
4,2025-01-25,0.0,2.893584,4.05834,-13.651576,3.553815,-3.94166,0.05834,0.0,-4.212634,...,-5.395801,-3.94166,0.05834,0.0,9.999803,4.05834,-13.651576,12.50343,-3.94166,0.05834


In [150]:
worksheet = spreadsheet.worksheet("net_approval_issues")
worksheet.clear()

set_with_dataframe(worksheet, net_issues.reset_index())

In [151]:
worksheet = spreadsheet.worksheet("generic_ballot_stats")
worksheet.clear()

set_with_dataframe(worksheet, generic_ballot.reset_index())

## Polls Display Tables

In [152]:
pres_polls['sponsors'] = '^Sponsor: ' + pres_polls['sponsors'] + '^'
pres_polls['sponsors'] = pres_polls['sponsors'].fillna('')
pres_polls['partisan'] = pres_polls['partisan'].replace({
    'DEM': ' (D)',
    'REP': ' (R)',
    'WFP': ' (WF)'
}).fillna('')
pres_polls = pres_polls.sort_values(['total_weight'], ascending=False)
pres_polls = pres_polls.rename({
    'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net'}, axis='columns')
pres_polls['Sample'] = pres_polls['sample_size'].astype(int).astype(str) + ' ' + pres_polls['population']
pres_polls['Start Date'] = pd.to_datetime(pres_polls['Start Date']).astype(str)
pres_polls['End Date'] = pres_polls['End Date'].astype(str)
pres_polls = pres_polls.reset_index()
pres_polls['Pollster'] = ('<a href="' + pres_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          pres_polls['Pollster'] + pres_polls['partisan'] + '</u></a>' + pres_polls['sponsors'])
pres_polls = pres_polls[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net', 'Adjusted Net']]
pres_polls

,Pollster,Start Date,End Date,Sample,Weight,Approve,Disapprove,Net,Adjusted Net
0,"<a href=""https://www.pewresearch.org/politics/...",2026-01-20,2026-01-26,8512 A,1.327445e-01,37.00,61.00,-24.00,-18.247610
1,"<a href=""https://echelonin.wpenginepowered.com...",2026-01-22,2026-01-26,1029 LV,7.078423e-02,41.00,57.00,-16.00,-18.318575
2,"<a href=""https://www.foxnews.com/politics/fox-...",2026-01-23,2026-01-26,1005 RV,6.969504e-02,44.00,56.00,-12.00,-13.304431
3,"<a href=""https://napolitannews.org/jobapproval...",2026-01-21,2026-01-28,3000 RV,6.754234e-02,48.00,51.00,-3.00,-12.993341
4,"<a href=""https://jlpartners.com/nationwide-reg...",2026-01-26,2026-01-26,1037 RV,5.791788e-02,40.99,50.91,-9.92,-16.908226
...,...,...,...,...,...,...,...,...,...
551,"<a href=""https://mclaughlinonline.com/pols/wp-...",2025-01-22,2025-01-27,1000 LV,2.636688e-20,52.00,43.00,9.00,5.571869
552,"<a href=""https://www.ipsos.com/en-us/reuters-i...",2025-01-20,2025-01-21,1077 A,2.290110e-20,47.00,41.00,6.00,9.946894
553,"<a href=""https://substack.com/inbox/post/15539...",2025-01-21,2025-01-21,742 A,2.037544e-20,49.00,36.00,13.00,8.312705
554,"<a href=""https://quantusinsights.org/f/februar...",2025-02-01,2025-02-03,1000 RV,1.988200e-20,51.90,45.30,6.60,-0.036917


In [153]:
worksheet = spreadsheet.worksheet("polls_display_table")
worksheet.clear()

set_with_dataframe(worksheet, pres_polls)

In [154]:
issue_polls['sponsor'] = '^Sponsor: ' + issue_polls['sponsor'] + '^'
issue_polls['sponsor'] = issue_polls['sponsor'].fillna('')
issue_polls = issue_polls.sort_values(['issue', 'total_weight'], ascending=False)
issue_polls = issue_polls.rename({
    'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net',
    'issue':'Issue'}, axis='columns')
issue_polls['Sample'] = issue_polls['sample_size'].astype(int).astype(str) + ' ' + issue_polls['population']
issue_polls['Start Date'] = pd.to_datetime(issue_polls['Start Date']).astype(str)
issue_polls['End Date'] = issue_polls['End Date'].astype(str)
issue_polls = issue_polls.reset_index()
issue_polls['Pollster'] = ('<a href="' + issue_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          issue_polls['Pollster'] + '</u></a>' + issue_polls['sponsor'])
issue_polls = issue_polls[issue_polls['Issue'].isin(issues)]
issue_polls['Issue'] = issue_polls['Issue'].replace({
    'economy': 'Economy',
    'immigration': 'Immigration',
    'foreign_policy': 'Foreign policy',
    'inflation': 'Inflation/prices',
    'trade_tariffs': 'Trade/tariffs',
    'healthcare': 'Healthcare',
    'crime': 'Crime'
})
issue_polls = issue_polls[['Pollster', 'Start Date', 'End Date', 'Issue', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net', 'Adjusted Net']]
issue_polls

,Pollster,Start Date,End Date,Issue,Sample,Weight,Approve,Disapprove,Net,Adjusted Net
0,"<a href=""https://static.foxnews.com/foxnews.co...",2026-01-23,2026-01-26,Trade/tariffs,1005 RV,2.209439e-02,37.0,63.0,-26.0,-24.214419
1,"<a href=""https://pro.morningconsult.com/tracke...",2026-01-23,2026-01-25,Trade/tariffs,2201 RV,1.334375e-02,43.0,48.0,-5.0,-17.563027
2,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2026-01-16,2026-01-22,Trade/tariffs,2226 A,1.131308e-02,34.0,57.0,-23.0,-24.296466
3,"<a href=""https://pro.morningconsult.com/tracke...",2026-01-16,2026-01-18,Trade/tariffs,2201 RV,6.752664e-03,45.0,46.0,-1.0,-13.563027
4,"<a href=""https://s3.documentcloud.org/document...",2026-01-09,2026-01-12,Trade/tariffs,1209 A,4.519375e-03,37.0,62.0,-25.0,-19.615968
...,...,...,...,...,...,...,...,...,...,...
1090,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-23,2025-02-25,Crime,1604 A,2.317930e-14,47.0,37.0,10.0,8.703534
1091,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-16,2025-02-18,Crime,1603 A,1.341444e-14,48.0,37.0,11.0,9.703534
1092,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-09,2025-02-11,Crime,1595 A,7.775951e-15,45.0,41.0,4.0,2.703534
1093,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-02,2025-02-04,Crime,1604 A,4.556612e-15,48.0,35.0,13.0,11.703534


In [155]:
worksheet = spreadsheet.worksheet("polls_display_issues")
worksheet.clear()

set_with_dataframe(worksheet, issue_polls)

In [156]:
gb_polls['sponsors'] = '^Sponsor: ' + gb_polls['sponsors'] + '^'
gb_polls['partisan'] = gb_polls['partisan'].replace({
    'DEM': ' (D)',
    'REP': ' (R)',
    'WFP': ' (WF)'
}).fillna('')
gb_polls['sponsors'] = gb_polls['sponsors'].fillna('')
gb_polls = gb_polls.sort_values(['total_weight'], ascending=False)
gb_polls = gb_polls.rename({
    'pollster':'Pollster', 'dem':'Democrats', 'rep':'Republicans', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net'}, axis='columns')
gb_polls['Sample'] = gb_polls['sample_size'].astype(int).astype(str) + ' ' + gb_polls['population']
gb_polls['Start Date'] = pd.to_datetime(gb_polls['Start Date']).astype(str)
gb_polls['End Date'] = gb_polls['End Date'].astype(str)
gb_polls = gb_polls.reset_index()
gb_polls['Pollster'] = ('<a href="' + gb_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          gb_polls['Pollster'] + gb_polls['partisan'] + '</u></a>' + gb_polls['sponsors'])
gb_polls = gb_polls[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Democrats', 'Republicans', 'Net', 'Adjusted Net']]
gb_polls

,Pollster,Start Date,End Date,Sample,Weight,Democrats,Republicans,Net,Adjusted Net
0,"<a href=""https://echelonin.wpenginepowered.com...",2026-01-22,2026-01-26,1029 LV,1.209856e-01,49.0,44.0,-5.0,-7.175952
1,"<a href=""https://static.foxnews.com/foxnews.co...",2026-01-23,2026-01-26,1005 RV,1.191240e-01,52.0,46.0,-6.0,-4.981116
2,"<a href=""https://e1.nmcdn.io/assets/ppp/wp-con...",2026-01-29,2026-01-30,652 RV,9.808209e-02,48.0,41.0,-7.0,-6.625918
3,"<a href=""https://www.claritycampaigns.com/clar...",2026-01-15,2026-01-22,1147 LV,6.493456e-02,51.0,42.0,-9.0,-9.337698
4,"<a href=""https://www.nytimes.com/2026/01/22/us...",2026-01-12,2026-01-17,1625 RV,6.023950e-02,48.0,43.0,-5.0,-5.648895
...,...,...,...,...,...,...,...,...,...
279,"<a href=""https://napolitannews.org/posts/gener...",2025-01-15,2025-01-16,1000 RV,3.718792e-20,44.0,51.0,7.0,3.369059
280,"<a href=""https://mclaughlinonline.com/pols/wp-...",2025-01-22,2025-01-27,1000 LV,2.677363e-20,40.0,48.0,8.0,2.936367
281,"<a href=""https://quantusinsights.org/f/trump%E...",2025-01-22,2025-01-23,1000 RV,1.331955e-20,45.0,48.0,3.0,1.082847
282,"<a href=""https://www.cygn.al/poll-musk-most-ef...",2024-12-09,2024-12-11,1500 LV,1.077667e-21,44.6,48.4,3.8,2.747353


In [157]:
worksheet = spreadsheet.worksheet("generic_ballot_polls_display_table")
worksheet.clear()

set_with_dataframe(worksheet, gb_polls)